In [4]:
# Class Neural Network

class NeuralNetwork:
    def __init__(self, inputs_nodes, hidden_nodes, outputs_nodes, learning_rating, bias):
        self.input_nodes = inputs_nodes
        self.hidden_nodes = hidden_nodes
        self.output_nodes = outputs_nodes
        self.lr = learning_rating
        self.bias = bias
        
        self.nscale = 1 / self.input_nodes ** (-0.5)
        self.w12 = np.random.normal(scale = self.nscale, size = (self.input_nodes, self.hidden_nodes))
        self.w23 = np.random.normal(scale = self.nscale, size = (self.hidden_nodes, self.output_nodes))
        self.b12 = np.random.normal(scale = self.nscale, size = (1, self.hidden_nodes))
        self.b23 = np.random.normal(scale = self.nscale, size = (1, self.output_nodes))
        
        sigmoid = lambda x: 1 / (1 + np.exp(-x))
        deriv_sigmoid = lambda x: (1 / (1 + np.exp(-x)))*(1 - 1 / ((1 + np.exp(-x))))
        
        linear = lambda x: x
        deriv_linear = lambda x: 1
        
        self.act_func = sigmoid
        self.deriv_act_func = deriv_sigmoid
        
        self.act_func_out = linear
        self.deriv_act_func_out = deriv_linear
    
    def show_fdebug(self, x, phi12, output):            
        print('w12', self.w12.shape)
        print('w23', self.w23.shape)
        print('w23_wo_bias')
        print('np.append(x, self.bias)', np.append(x, self.bias).shape)
        print('phi12', phi12.shape)
        print('output', output.shape)
        print('b12', self.b12.shape)
        print('b23', self.b23.shape)
        return None     
    def show_bdebug(self, hidden_error, output_error_term, hidden_error_term):
        print('hidden_error', hidden_error.shape)
        print('output_error_term', output_error_term.shape)
        print('hidden_error_term', hidden_error_term.shape)
        return None    
            
    def shuffle_data(self, features, targets):
        df1 = pd.DataFrame(features)
        df2 = pd.DataFrame(targets)
        df = pd.concat([df1, df2], axis = 1)
        df = df.sample(frac = 1)
    
        features = df.iloc[:, :features.shape[1]].values
        targets = df.iloc[:, features.shape[1]:].values
        return features, targets  
   
    def feedforward(self, x): 
        
        phi12 = self.act_func(np.dot(x, self.w12) + (self.bias * self.b12))
        dphi12 = self.deriv_act_func(np.dot(x, self.w12) + (self.bias * self.b12))
        output = self.act_func_out(np.dot(phi12, self.w23) + (self.bias * self.b23))
        doutput = self.deriv_act_func_out(np.dot(phi12, self.w23) + (self.bias * self.b23))

        #self.show_fdebug(x, phi12, output)
        return phi12, dphi12, output, doutput
    def backforward(self, phi12, dphi12, output, doutput, error):
        
        output_error_term = error * doutput
        hidden_error = np.dot(output_error_term, self.w23.T)
        hidden_error_term = hidden_error * dphi12
        
        #self.show_bdebug(hidden_error, output_error_term, hidden_error_term)
        return output_error_term, hidden_error_term   
    def update_weights(self, x, phi12, output_error_term, hidden_error_term, dw12, dw23, db12, db23):
        
        dw12 += hidden_error_term*x.reshape(self.input_nodes, 1)
        dw23 += output_error_term*phi12.reshape(self.hidden_nodes, 1)
        
        self.w12 += self.lr*dw12/features.shape[0]
        self.w23 += self.lr*dw23/features.shape[0]
        
        
        db12 += hidden_error_term*self.bias
        db23 += output_error_term*self.bias
        
        self.b12 += self.lr*db12/features.shape[0]
        self.b23 += self.lr*db23/features.shape[0]
        
        return None
  
    def train(self, features, targets, epochs):
        for e in range(epochs):
            features, targets = self.shuffle_data(features, targets)
            error_rms = 0
            dw12, dw23 = np.zeros(self.w12.shape), np.zeros(self.w23.shape) 
            db12, db23 = np.zeros(self.b12.shape), np.zeros(self.b23.shape)
            for x, y in zip(features, targets):
            
                phi12, dphi12, output, doutput = self.feedforward(x)
            
                error = y - output
            
                output_error_term, hidden_error_term = self.backforward(phi12, dphi12, output, doutput, error)
            
                self.update_weights(x, phi12, output_error_term, hidden_error_term, dw12, dw23, db12, db23)
                
                error_rms += np.mean(error**2)
                
            Total_error = error_rms/(2 * features.shape[0])
            
        return print(Total_error)
        
    def run(self, features_run):
        aoutput = []
        for x in features_run:
            phi12, dphi12, output, doutput = self.feedforward(x)
            
            aoutput = np.append(aoutput, output)
        return aoutput
            

In [5]:
import numpy as np
import pandas as pd
import random as rd
import matplotlib.pyplot as plt

np.random.seed()

n_records = 5

features = np.random.rand(n_records, 4)
targets = np.random.rand(n_records, 2)

#features = np.random.rand(n_records, 1)
#targets = 10*features**2

hidden_nodes, learning_rate, epochs, bias = 3, 0.4, 5000, -1

nn = NeuralNetwork(features.shape[1], hidden_nodes, targets.shape[1], learning_rate, bias)
nn.train(features, targets, epochs)


#plt.scatter(features, targets)
#plt.scatter(features, nn.run(features))

#plt.show()

0.00241070096572
